In [41]:
#!/usr/bin/env python
"""
Firmware Updater Tool

A frame consists of two sections:
1. Two bytes for the length of the data section
2. A data section of length defined in the length section

[ 0x02 ]  [ variable ]
--------------------
| Length | Data... |
--------------------

In our case, the data is from one line of the Intel Hex formated .hex file

We write a frame to the bootloader, then wait for it to respond with an
OK message so we can write the next frame. The OK message in this case is
just a zero
"""

import argparse
import struct
import time

from serial import Serial

RESP_OK = b'\x00'
RESP_ERR = b'\x01'
FRAME_SIZE = 16
PACKET_SIZE = 1064

error_counter = 0

def send_metadata(ser, metadata, nonce, tag, debug=False):
    version, size = struct.unpack_from('<HH', metadata)
    print(f'Version: {version}\nSize: {size} bytes\n')

    # Handshake for update
    ser.write(b'U')
    
    print('Waiting for bootloader to enter update mode...')
    while ser.read(1).decode() != 'U':
        pass

    # Send the metadata to bootloader.
    if debug:
        print(metadata)

    ser.write(metadata)
    ser.write(nonce)
    ser.write(tag)

    # Wait for an OK from the bootloader.
    resp = ser.read()
    if resp != RESP_OK:
        raise RuntimeError("ERROR: Bootloader responded with {}".format(repr(resp)))

def send_frame(ser, frame, debug=False):
    ser.write(frame)  # Write the frame...

    if debug:
        print(frame)

    resp = ser.read()  # Wait for an OK from the bootloader

    time.sleep(0.1)

    if resp != RESP_OK:
        raise RuntimeError("ERROR: Bootloader responded with {}".format(repr(resp)))

    if debug:
        print("Resp: {}".format(ord(resp)))
        
    #If the bootloader receives a one byte, resend the frame and increment error counter
    if resp == RESP_ERR:
        error_counter += 1
        send_frame(ser, frame, debug=debug)
        
# Open serial port. Set baudrate to 115200. Set timeout to 2 seconds.

firmware_blob = b'\xbc\n\x16\x16\x16\x04\x04\x04\x8b3\xdd;2\xebKW\xf2\x0fy\xedoU\x07Q`4K\x17\xcf\xbd\x92\xa6\xe9\xf4^\xeaj\xb3\x9f\xc2\xcftl\x82\x13\x8dF\xab\xc0t\x07\x82XZm\xd1\xc0\xc5o\xd4\xefK\xe0(q\xa8\x03\xa5!\x91\x05\xbfA\x06\x8b\xde\xca\xe8\xe9\x02\x00^\xa2\x18\xa6@h[\xea\x84!\xfc|\x83\xff\xd2\x8eM\x9d\xa7\x1b\x82\xf4\x83(\xd4\xc8\xf3\n\xab\x9a4\\\x12|s[1\r\x16\x83B5\xf4\x07\xe9j\x0c\xfc\xd8@E\xb4l6\x1d\xaa\xab\xf6^\\\xb9\xdd\xfb\xdau\xe9\xce\x81\xfbi\x92\xaf\xdb\x11\xa5uO(x\xeb\xe2n\xae\xd9\x87n*\xf0v\xfa\x12H\x1e3ALYv%\x11 \x02&J\xcdr\x8d\xc9\xfa\xe2\x0c\xb4Q\xafQJ\x87c\xdd0\x92\x93\xb6W\xd1{c\xaa\xf29c\x97\xb3\x8b\x92q\xb5\x17\x1cK\x15I\xc2\xee\xb5 \xd6Y\xab\xd2\xcf\xe8\xc6\x9d\'J\\D&\xfaM\xab|O\xd0\xfd\xb65\xd9\x16\xa0\xd6|\xcew\x8a\x83\xd2=\x90\x0b\xedm\xae[\x82@c~\xe2\xbd\x16\x10\x12\xe19\xd4\x92\x91]\xa6\x87\xed\xda\xc8S\x08%\xb0\xfa\xf6\xb8\x12f~X\xbfSd\xadW0\'\x173O\xa2\xb0$\x14\\6\xeb\xc3\xd5\xf1\x89I\xe8\x97\xfdF\x0f\xc9C\x86\xd0\xd7x\xee\xd6\x8eM9^I{]*}\xb8:\xfd \xe3\x9a\xe1@\x8c[\xecl\xe7k\x1a\xbf\x01j\x05\xf2\\\x97"\xbc?\xc6\xfd\xa7\x01\xd3\xc3\xf4\x9f\xde5\xf7\x17\xfd\xb1\x04\x8c\x9dY\xbc[\xcf\xf5\xd3\xbb\x95\x1e\xa4\xd9\xc7\xbf4\x1c\xb8\xe6\xa6\xc1\x91\xeb\xa7\xdeMR\xad\x1a\xf0\x01v8k\x00\xc7%C\xfeu\xdc\xa4\x16\xf2LM7\x9fVj\x12\xef7ljsn,\x8e.`\x01 \xc4\x81:\xe5\x16[\x0f\xe2o\xdb:\xcd\xff\xcd\xfe\x96\x1d\xcd\x8a\xdc\xf0\x97n\xa2\x89\x91\xe8\x12r\x87\xbb\\\x00\xb9\x85\x95\x93\xee\xe3<\x05l\xea\xb2\xb2\xb9@O\x9e\xb5p+\xb9(\x84\x9d\'\x81JL.K\xcd1\x16\x9b\xb6G\xd4\xc5\xbe\x90\\m\xa8\xa41\x9d"\x80\x1c\x85\xd3~\ny\xd3+\x7fI\x91?\x7foqK\xc9X! \x0e\xf0\x8el\x145\x9bu8\x82\xe8\xc0\xe4\x82\xb8X@\xd0E\x03#\xe3\xb4G\x95o\xe9\x11-L\x85\xa4}M\xbb\xb6\xa8\xf9\x00F\xe63Oc\xc8\xb7s\xf6\xc7\xf8\xce\x85V\x00\x8e.\x9f\xdb#\xdb\xd5\x17\xba\x0e\x06b\xcc.\xddu\xa5\x03\xe2rQ\xca\x05I\x1bQ\xe6\xb2\xf1\xce\x89\xe9%\x14UP!\xf5\x85\x19o\'\xbeP\xf96H\xfe7qDc\x85\xb9\x9a\x99)z\xc1A*g\x92\xd4\xd0\xab\x06\x86\xec\xb4\xc7\xeb\xca\x0ciE\x7f(\xc7:!\xceAt\xf3f\xa3#\x00,\xfa\xa1A\xd4\xbdJ_\xa7\xa44T)\xa4*\x8d\x89\xd2\xa6\xf6\xca\xd1\xf3\xcef\x05\xc9\x86\xfc!\xd9\xdf^\x01\x8bZ\\\xca\xf2r\x0en\x89\rS\x079\x9b\x8e\x101\xf9\xd7\xe5\xecS\xf4\xc1}Nxx\xe4d\x15|\xe3\x9b\xf18\x84j6\xd9D\xd4P\xe2O\x0c"5/\xf9\'4,A6\xad\xf8\xbau\xb6\xbe\x08\xd8I\xbf\x10\xf8\x05r\x1c\xa9vy|\xfb\xc8\x84\xb9\xce\xf7\xd8CZo\xe3n\xb5\xfd8sT4\xfa\x01\x01/\xf8\xca\\\xa3\xcd_Kw\xe7\x87\xd78\xdd\xf9L\xa5R\x9dZ`\x16 $\xa3kF\x0e\xa0I\xe9A]\xaacg\x84H\x8c6\xaf\xe3\x9c\xfa\xd2y\x85\xb0Y\xdf\x93N\x0eW\x8dY\xf6\xeca\xa6\xd1$\t\x9f\x90q\x83\x1f6\x0b9\x10\x11\xbcU\xb5\xdbQ\x13\xe0&v\x01G\x05\x15\x1bD\x85\xa4\nN\xb5DG\xf3\x98\xdb5\xd9h\x9d\xd8\x84\xf4#\xde\x9a+x\xd9\xb6\xf4\x9b(c\xb2y}\xb2\xb1\'\xa8\xaf\xfc\x84\x14\xd69\x96\xa6\xf6,\x842_\x9b\xd4q5\x8ca\xc17O\x19\xf1\x98!\x0fj\x94\x10\xe1\x0c\xa0:\xde\xaa\x95\x88\xaf\x04\xa1\x9a:\tx\x8e\xc1\xd7\xcaK-)\x96\x0bKq\x91\xc6R\xdbL\xa0.[\xa0\xb56z\xa6\x16\xd99%}\xef;\xe0\x17Z\x12\x11\xf0\x95=(\xd2 \xf55n\x00\x0e\x10hzb\xed6~\x19\xe6\xba-\xde\x95\xed\xdc;\xb7]\xd8\xa1H\x98U\xc2B\xb0P\x00\xca\xf3b\x11\xcf\xbc\n\x04\x00\x00\x04\x01\x00\xd0[e\'\xfe\xc9\xce\x93\xc2p\xadX\x02\x81\xec\xa4\xae\xaf\xbd\xca\x7f\x9a&\xefG\x99%\xdb\t\x88y$\x99\xaf\xfa\xbbwP2\x8b_\xf4G\x0c\x01;\x8e\xaa$\x11\xb6\x0bj\xcc:\xd4\xd4\xa1\xaa\x12)\x9a-\xe62\xc9\x152\x19\x13J\x06!#+<;\xc7\x8c\xb6\xeesAN\xe6\x10\xfa\xdb\x08\x16\xec\x89\xc3\x07D\x1a\x0c\xef\xb4\xaa\xa7\xceW\xdbm\xdf\n\xe7\xf3y\x88\x8b\xf1D\xea\xe5\xd26\x8e: \x16\xf7\x16\x1d\xdeym\x16\xac,\xac^2dU\xc8`\x1b2Z\x83/\xc4\xfb^\xe3\xc1|kw\xfdw\xeev"=\xd0\xe5\xea\x00\xd1dW\xdc\xa8\x08g\x86\x16\xfey\x96\x9b\x0e\xbe\xa6_\x15\x16\x8e}\xebk\x08\xe7%(\xfc/\x82o\x0c\xfc\xf1K\xcc\xf8\x84\xf2\x8f\x1f\x95=V"!\xe5A\x90\xb7g\xb9\xf9\x07\x93J\x1c\x8a\xa6oRm\xa9,"\x81-I?\x8c#\x7f\xab\x07\xf9B\xc0\x1c3N*\x17\xe3Q\xcfK\xbee\xa4\xedy\x88\xff\x18\xe0\xcc\x86\x89\xc3g|&\xf5V\x11\xcf\xf6\x11\xec\xc0!\t\xb2f\x94\x84\x8c1D\xdcVs*\xb8~\xb1\xce.\xf3\xf2\x81|\x0b6\xb3\xfa\xfasX\xfc\x80UH?@\x1e\xc8#Y\x12[\x9c\xaf7\xc5\n\x96\xd6\xf3X\x89\xf3\x1d\xe4\xd6\x14f\xf0\xd74XC\xf7\xa8g\xe1\xe4\'Y8\x97\x81\xe7\xaf\xbf\xf0!eR\x145\x08dSY\x15@\xb5\xf2k]\xd3\xf7\xc0\xd5(\xdb\xb0\xb0;\xca/~\xae\xe6\x86\x9c\x87 \xe1A\xeb\xdbV\xda;Pf\x97J2\xf5O\xc9_\xe9s\xdb\xc4*\xc5\xe2\xc0\x05\xdf\x16$\x8d\xd2\x99a\xf7`\xc8@u9\xeb$\xbcsP\x9ajwo\x81\x86\x1bP\xb8\x7f\x18ua|C\xce\x8e\xae\x0e\rD\x01\xb9A7B\xf4\xe5\xd3\xf4T\x1at\xe0K\xb3\xc0>\x08\xed&S\xd5L4\xfc_\xb6G\xb9\xf2\xd7Q\xdb\xf7\xdd\x85UM^\x14\x85\xa8=\x02\xc1\xa2\x1e\xdb\x1c\xc5\xccW\xe2\xde\x05l\xcc4\xba\x9d\xa5\xec\x95/;\xb3\'\x1b\x04\xc5\xe46\xfb[\x1b:\x9e[\xfc\xfc\xff\xb8I\x18}m\xcf\x96\xba\xb5\xd1lH\xa4\xf6_\xc0\x92.1\x16\x1d\x83\xe2\x96\x95\x9d\xe2V\x00\x83q\x0c\xa4\x0c\x1c\xe8\xcen\xf7\xc1U\x97\xe38\xd8\x19\tp5\xc0\x02\x7f*\xf5i-{ Ee\xf9x\xca\xbb\xd5$\xad\xd6\xd9arE\xaf\x9dB\x98\xa7\xff\x12gi\xd9<\x1c\xfc\xefj\xb9\'\x92\xcc6H4\xc1|\xef1\x97Y\xd6\xbe\xf0_\x01\xea\xd3\x15\x04R\x148\xf6\\a\xd9o3{\x9d\x02\xedL=\xae\xbe\xec\xa5s\xff\x06\xa7\xec)\x06\xa2\xa4;\x119\xd6\xe94\xd1\x93wsE\xec?\xdbG\xee\x14u\xdc\xabu\r\xf2\xdd\xc3\x85#\x03\xd29k\xb3\xc1\xd1b\xaf\xff\x07\x83K\x8d\xe2I\xecK"\xec\x009\xaf\x8b2\xb8\x9c\x81Dq\xa7i\xde\xe6\xd8\xc1\xa9W\xe8\xbf\xeaI\xa5jY\xb8\xa2AT\x1f\x8f_>\xad\x17!"t\xd7q\x08\xf1rK\xf0\nV\xe0I\x02\xb3\xd1W+\xd7\xba\xf2\xf2\xeb\x99\xd7\x04\n\xe31\xd3\xc4W\x149d\xa1\xae\xba\xe1{\xc3z\xba\x13\xde\xe2\xe3\xec\xef{<\xa0\x92\xc7\xaeu=\xd6\xfb[$\xa90\xcf\xa8\xb3!\x1aU\xad\xba\xf8\x8e^AJ\xe9\xe9|\\v\x8e\xa7}4\x03\'M\xf4\xc5\xef;\xde\xceO\x97\xb3\xc9\xce\x18\x9f\xfc\x9e\x88\x85\x1f\xa6\xdc\x19Id#\x86!\xefu"2\x8b\xe8\x8c\x860\x8e\xd2#\x10P%\xf9@G\xc7\xc7\x9f\x91K\xdf=\xc9\xb8c"|?\xd7F\x94>\n\x00~\xa2\xf6"\'%\xbb\xf8n\xac\xe7b\x0cl\xb6\xcd\xf3v/\xc7\x054O* \x18\x0e\xe3F\xe8\x0b\xff/U\xfc\xd4\xbf\xb8\x96\x08T\xdbc\xc5\x8c&;\xb0\x8d\x8a\xc1,\x17\xab\x9b\x1e\xe8\xfe\xd8f\x8fB\xed\nV\xef(\x11\xa89\x07\xc0\xc9\xb7\xda\x83\xaez\x05\xba\xc5\x82\x0f\x9dn@\xe5<\xa4\xea\xfa\xbb\x8c\xf8nD\xae\xbd\xd0\x16\xc0PZI\xef\x98\xd9\xc6$\xb7\x1d\xf9\xc3\x05.$\x07N\xbe\xff>\xa9-\xa2\xc28\x04(\xa1\xdbt\xb7\xe9g; \x11\xbc\xb9]\xc8q\xc9\xf4}\x7f\x9b\xf4\x18\xa8=i\xbc\xc6&\x1bx.\xfaf\xbc\n\x04\x00\xbc\x02\x02\x00\x1d\x03\xef\x8c\xed\xf6k\x15SRC\xd4\xab\xf8\x04\xc11\xf9k\xba\xb8!}\xf5\x9b\xa9\xe0N\xdd~\xb4\xf3\xe0\xe8Y\x85\x00\x12\xeaF=\x8909T\xbb\xbb\xb0v\xe3\xc4\x0b\xe0\xcbf2Q\xdb/\xd3:7J\x9e,uo4o\\\xfe~\x8a+\xdd\xcf\xe4\x18\xb8\xd5\xa52E\xba@M&R\xac\xec\x03F\xda6O+\xb4\xb0S\x04\xc1\x18\xaf\xbfs\xb8\x93\x95\xed\xac\r\x93\x19\xa8G\xa0\xda\x93|U\xeco\xe9\xeb\xac\xe8\xeb\x97\x95\x0f\x07\xb1rQ\x186\xe9/_\x98.!r\xf5\x07\xaaA_\'\xf1bi\x18\xc0\xa3Tgg\x9b\x14\x9c\xbc\xcd\xe5=\x9f\x1a\x86\x90\xb9\x82\xdaz\xdfs\xcc\x161\x86\xc9\x06.X`\xe7\xa9}4\x99\x94\xd3[\xcf2M\x92n\xbdG\xfd \xe6\x0c\x9e\x0fba\xed-\x9e(ke\xa8`\xf6\xdd\xd9\xda[\xcci+\xc4?\xd1D\xda5\xf8\x9c\x06\xfd\x1d_0\xa1\xed\xc7\xe6\x0c\xe1 \xf2C@\xad\xcd\xe2\xf7\xeeY\xc0\xca\xc4"g\xf62\xa3\xc7\xbf\'\xc3\x0c\xba(\x8c\'2\xc4\x8d\xdeF\xfb73\xc0U\xfc\xcf\xb6\x87\x14\xe9\x08\x84P{.Q\x8b\xd2\xc8r\xb9\x05\xd4\xdc\x178\x9d\xd8\x9d\xe2 }\xda\x8f\xfb\xe5\x06\xe1c\xf3E\x88K\x9e\xd1\xdf1\xe8\x9e9J?<\x97\x8d\\W>N\xda\\,\xf2HH\xd4\x8d\x90\xb1z\x08\xdak!\xc0\x16n\xdfF\xdc\xc3\xceS\x95\\Z\x1b\xd1\x85i\xf8\xc1\x1a\x12\xb1\xc3\xde\x07\xb39P\xe5\xbdx\x82\x896\x83E\xd8\xfd[\x950\xc7\xf9y|\xfa4\xcaD\xe6"\xbc7\xca1\x14]\x98@c&5\xc7\xae\xed\x1b\x12\xaf\x82\xe0e\xf6\x1c`\x8cF\xb1\n\xe1\x17\xae\xd5\xff\xfd\x1a\xb9pRGN\x12jL\xee\xdd\x9a\xecF\x9e\xa0\xb4\x04\x93p\xbf\xa0\xf5\xab{\xf47\xbdL\x04\x8ca\x81\x07\x9f\xe3\xea\x9c\xee\x03<H)^\xcf\x9c\x89\x8bZUk\xd2\x1e\x8c>\x13\xe7UB\x03\x9b\xbeB\xb6\x86;w\xd9O\x1f\xedfOa\xe9#\x07\xee\xb3xjhWPk\xc2 y\xd7\x95\xbc\xdemz\x9d\x90\x98zM\x01\xc5"\xf0MO\xd5\x00!v\xd3d\x0e\xb3E\xb8\x8f\xe2\x9c\x19\xd9\xbdx\x80\x98\x84\xd4\x99\xa9\xc3+\x97\xc8\x066\x9e\xca\x85\x93\xed\x96\xfcNk\x12\xb9J\x9fJ:\x87z5\xb3\xc9A\xe8\x98Q\x87\xb7\xc4\xe2\x84\xa4\xa2\x8bq\xd38\x8c\xfe_jLn\xe6\xc61&\xb5o\xca\xd6R\xaf\xaa<=Y\x88\xf1\xbb\xa8\x8bizH\xd6\x8d\xdc\xef\xe2\x8a\xf1\xadJ\xe1\xb1&\x80\x12\xc2\xdf\x03\xc5\x82dy\xb2u)wpr\x88N//&\xc7\xc3jd{t\xa84Y\x84qko\x14@2\x19\xf1\xea\x7f\xd6\xa7}W\x0e\xcb\x80\xf5\xaaJ\x85Z\xac\xd4\xd4\x9d\x07\xf6\xc9\x11|\xe17\xd1\xc1\xd9\x15\xc3#\x82>?\xfa\xbb`\xa7\xb6\xb4\x12-\xeb\xcf\n\xc1t\xe8x\xd9\x94\xdb\xffA\xa2b\xd4\x03\x00\x04\x00\xbc\n\xff\xff\xb5Z\x13\xcf\xfb\xc4\x7f\x956\xa2-.$\x8c\xe3*\xd8\x8a#\x89\x14\xa6\xcbs\xc3\xa0\xfc\nz90A\xf0\xbfg\xb14\x84\x87\xecvW\xa0,\xe0\x8d\xd7\xdb'
metadata = firmware_blob[:8]
nonce = firmware_blob[8:24]
tag = firmware_blob[24:40]
    
#send_metadata(ser, metadata, nonce, tag, debug=debug)

fw_size  = struct.unpack('<H', firmware_blob[2:4])[0]
chunk_size = struct.unpack('<H', firmware_blob[6:8])[0]
num_chunks = int(fw_size / chunk_size)
packet_index = struct.unpack('<H', firmware_blob[4:6])[0]

error_count = 0
    
for i in range(0,num_chunks):

    fw_start = PACKET_SIZE*packet_index +40
    firmware = firmware_blob[fw_start:fw_start+1024]

    for idx, frame_start in enumerate(range(0, len(firmware), FRAME_SIZE)):
        data = firmware[frame_start: frame_start + FRAME_SIZE]

        # Get length of data.
        length = len(data)
        frame_fmt = '<H{}s'.format(length)

        # Construct frame.
        frame = struct.pack(frame_fmt, length, data)

    #If there are more than ten errors in a row, then restart the update.
    if error_counter > 10:
        print("Terminating, restarting update...")
        #return 

print("Done writing firmware.")

b'\x16\x04'
1046
1046
1046
1046
1046
Done writing firmware.
